In [ ]:
#export
import numpy as np
import torch
import pickle
import io
import torch
from copy import deepcopy
from fastcore.all import *
import matplotlib.pyplot as plt

In [ ]:
# default_exp utils

In [ ]:
#hide
from nbdev.showdoc import show_doc
from nbdev.export import notebook2script
%load_ext autoreload
%autoreload 2

# Utils

> Definition of utility functions that deal with simple but recurrent transformations.

## Training utilities

In [ ]:
#export
def plot_trainings(training_results, eval_optims=True, expl_optims=False):
    "Plots the training results."
    use_legend = eval_optims or expl_optims
    mean_rewards = np.mean(training_results['rewards'], axis=0)
    std_rewards = np.std(training_results['rewards'], axis=0)
    mean_params = np.mean(training_results['params'], axis=0)

    fig, ax1 = plt.subplots(figsize=(10, 5))
    color1 = "tab:blue"
    ax1.set_xlabel("episodes", fontsize=18)
    ax1.set_ylabel("Reward", color=color1, fontsize=18)
    ax1.plot(mean_rewards, alpha=0.8, label="Rewards")
    ax1.set_ylim([-0.05, 1.05]); ax1.set_yticks(np.arange(11, step=2)/10)
    if eval_optims: ax1.plot(np.mean(training_results['eval_optims'], axis=0), alpha=0.8, color='C3', label="Optims eval")
    if expl_optims: ax1.plot(np.mean(training_results['expl_optims'], axis=0), alpha=0.8, color='C4', label="Optims expl")

    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid()
    if use_legend: plt.legend(loc='best')

    ax2 = ax1.twinx()
    color2 = "tab:orange"
    ax2.set_ylabel("Parameters",color=color2,fontsize=18)
    ax2.plot(mean_params,color=color2,alpha=0.5,label="Params")
    ax2.tick_params(axis='y', labelcolor=color2)

    fig.tight_layout()
    
def arrange_shape(obj, mode='expand'):
    "Makes all lists within a list to have the same length. Can be mode 'cut' or 'expand'"
    rows = len(obj)
    if mode == 'expand':
        cols = max([len(obj[k]) for k in range(rows)])
        M = np.zeros((rows, cols))
        for r in range(rows):
            list_length = len(obj[r])
            M[r, :] = obj[r] + [obj[r][-1]]*(cols-list_length)
    elif mode == 'cut':
        cols = min([len(obj[k]) for k in range(rows)])
        M = np.zeros((rows, cols))
        for r in range(rows):
            M[r, :] = obj[r][:cols]
        
    return M

def best_so_far(m):
    "Returns best value up to each point along the second axis of a matrix for each row."
    m = np.array(m)
    M = np.zeros_like(m)
    for k in range(M.shape[1]):
        M[:, k] = np.max(m[:, :k+1], axis=1)
    return M

In [ ]:
#export
def convergence_time(results, tol=5e-4, T=50, t_avg=20, return_diffs=False):
    "Returns the convergence with criterion of not changing result by `tol` for `T` epochs."
    if 'training' in results.keys(): results = results['training']
    rewards = arrange_shape(results['rewards'])
    mean_rewards = np.mean(rewards, axis=0)
    epochs = len(mean_rewards)
    moving_avg = np.convolve(np.array([1/t_avg]*t_avg), mean_rewards, mode='valid')
    diffs = np.abs(moving_avg[1:] - moving_avg[:-1])
    diff_variation = np.convolve(np.array([1/T]*T), diffs, mode='valid')
    try:    t = np.where(diff_variation <= tol)[0][0]
    except: t = len(mean_rewards)
    if return_diffs: return t + 2*T, moving_avg, diff_variation 
    return t + T

@delegates(convergence_time)
def indiv_convergence_time(results, max_epochs=800, **kwargs):
    "Similar to `convergence_time` but with each agent."
    results = results['rewards']
    times = [convergence_time({'rewards': [res[:max_epochs]]}, **kwargs) for res in results]
    return np.array(times)

def get_indiv_times(tl_eval, convergence_crit=None):
    "Provides convergence times from individual ratios."
    default_crit = {'T': 50, 't_avg': 100, 'tol': 2e-4}
    convergence_crit = {**default_crit, **convergence_crit} if convergence_crit is not None else default_crit
    Bs = list(TL_evaluation.keys()); Bs.sort()
    time_ratios, time_err = [], []
    for b in Bs:
        ts_tl = indiv_convergence_time(tl_eval[b]['tl'], **convergence_crit)
        ts_0 = indiv_convergence_time(tl_eval[b]['vanilla'], **convergence_crit)
        t0, ttl = ts_0.mean(), ts_tl.mean()
        ratio = ttl/t0
        ratio_std = np.sqrt((1/t0)**2*ts_tl.var() + (ttl/t0**2)**2*ts_0.var())
        time_ratios.append(ratio)
        time_err.append(ratio_std/np.sqrt(len(ts_0)))
    return Bs, np.array(time_ratios), np.array(time_err)

## Load and save data

In [ ]:
#export
class CPU_Unpickler(pickle.Unpickler):
    "Unpickles from GPU to CPU"
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [ ]:
#export
def save_benchmark(benchmark, N, H, method, suffix=None):
    bench_dir = Path("../benchmarks/"); bench_dir.mkdir(exist_ok=True)
    file_name = f"bench_N{N}_{method}_{H.model}_B{state2str(H.linear)}_J{state2str(H.quadratic)}"
    if suffix is not None: file_name += f"_{suffix}"
    save_path = (bench_dir/file_name).with_suffix(".pkl")
    with open(save_path, "wb") as f:
        pickle.dump(benchmark, f, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_benchmark(N, H, method, suffix=None):
    bench_dir = Path("../benchmarks/")
    file_name = f"bench_N{N}_{method}_{H.model}_B{state2str(H.linear)}_J{state2str(H.quadratic)}"
    if suffix is not None: file_name += f"_{suffix}"
    load_path = (bench_dir/file_name).with_suffix(".pkl")
    with open(load_path, "rb") as f:
        benchmark = pickle.load(f)
    return benchmark

def load_checkpoint(H, maxP, ID, episode):
    ckp_dir = Path("../trained_models/checkpoints/")
    ckp_name = f"ckp_N{H.N}_{H.model}_{maxP}_id{ID}_e{episode}.pt"
    return torch.load(ckp_dir/ckp_name)

def checkpoint2results(N, model, maxP, IDs, episode):
    "Loads checkpoints from IDs and converts them to result format."
    models, envs, final_rewards, final_params, final_energies, final_optims, expl_optims = [], [], [], [], [], [], []
    visited_states, visited_energies, visited_params, oracle_rewards, visited_rewards = [], [], [], [], []
    for ID in IDs:
        try:
            ckp = load_checkpoint(N, model, maxP, ID, episode)
            models.append(ckp['model']); envs.append(ckp['env']); final_rewards.append(ckp['final_reward'])
            final_params.append(ckp['final_params']); final_energies.append(ckp['final_energies'])
            final_optims.append(ckp['eval_optims']); expl_optims.append(ckp['expl_optims'])
            visited_states.append(ckp['visited_states']); visited_energies.append(ckp['visited_energies'])
            visited_params.append(ckp['visited_params']); oracle_rewards.append(ckp['oracle_rewards'])
            visited_rewards.append(ckp['visited_rewards'])
        except: print(f"Failed to load ID{ID}")
    training_results = {'models': models, 'envs': envs, 'rewards': final_rewards, 'params': final_params, 
                        'energies': final_energies, 'eval_optims': final_optims, 'expl_optims': expl_optims}
    exploration_results = {'agents': models, 'envs': envs, 'visited_states': visited_states, 'energies': visited_energies, 
                           'params': visited_params, 'oracle_rewards': oracle_rewards, 'visited_rewards': visited_rewards}
    return {'training': training_results, 'exploration': exploration_results}

def save_model(agent, H, maxP, ID):
    agents_dir = Path("../trained_models/")
    agents_dir.mkdir(exist_ok=True)
    if H.model == 'xy':
        agent_name = f"agent_N{agent.N}_{H.model}_{maxP}_id{ID}_{state2str(H.linear)}_{state2str(H.quadratic)}_g{H.g}.pt"
    elif H.model == 'graph': 
        agent_name = f"agent_N{agent.N}_{H.model}_{maxP}_id{ID}.pt"
    else: 
        agent_name = f"agent_N{agent.N}_{H.model}_{maxP}_id{ID}_{state2str(H.linear)}_{state2str(H.quadratic)}.pt"

    torch.save({'model': agent.model, 'state_dict': agent.model.state_dict}, agents_dir/agent_name)

def load_model(H, maxP, ID):
    agents_dir = Path("../trained_models/")
    if H.model == 'xy':
        agent_name = f"agent_N{H.N}_{H.model}_{maxP}_id{ID}_{state2str(H.linear)}_{state2str(H.quadratic)}_g{H.g}.pt"
    elif H.model == 'graph': 
        agent_name = f"agent_N{H.N}_{H.model}_{maxP}_id{ID}.pt"
    else: 
        agent_name = f"agent_N{H.N}_{H.model}_{maxP}_id{ID}_{state2str(H.linear)}_{state2str(H.quadratic)}.pt"
        
    return torch.load(agents_dir/agent_name)

## State transformations

In [ ]:
#export
def simplify_layout(L):
    "Simplifies the given layout of the form `[np.array([0, 1]), np.array([0, 1, 2])]`."
    to_simplify = []
    for k1, sites1 in enumerate(L[:-1]):
        if k1 not in to_simplify:
            for k2, sites2 in enumerate(L[k1+1:]):
                overlap = len(np.intersect1d(sites1, sites2))
                if   overlap == len(sites1): to_simplify.append(k1); break
                elif overlap == len(sites2): to_simplify.append(k1+1+k2)
    return [sites for k, sites in enumerate(L) if k not in to_simplify]

def fill_layout(L, N):
    "Fills layout with single-body terms that are missing."
    for n in range(N):
        included = any([(sites == n).any() for sites in L])
        if not included: L.append(np.array([n]))
    return L
    
def state2int(state):
    "Takes state vector as binary encoding of an integer."
    return int(state2str(state[::-1]), 2)

def state2str(state):
    "Takes state vector and outputs a (spaceless) string."
    return ''.join(str(int(i)) if isinstance(i, (int, np.int64)) or i.is_integer() else str(i) for i in state)

def state_in_list(state, state_list):
    "Checks whether a state vector is in a list of states."
    return any((state == x).all() for x in state_list)
    
def T(state):
    "Takes state and converts it to `torch.FloatTensor`"
    return torch.FloatTensor(state)
    
def flip(state, i):
    "Flips constraint `i` from state."
    flip_state = deepcopy(state)
    flip_state[i] = -flip_state[i] + 1
    return flip_state

def contained_constraints(state, N):
    """Provides a boolean mask indicating the constraints already contained within a larger one in 
    the given state. 
    Each group larger than pairs contains 2 of the immediately smaller groups, 3 of the next,
    4 of the next and so on. For example, a group of 5 will contain 2 groups of 4, 3 groups of 3 and 4 groups of 2."""
    mask = np.zeros(len(state), dtype=bool)
    ii = np.where(state[N:] == 1)[0] + N              
    for i in ii:
        steps, res = i//N, i%N                   
        for j in range(steps):
            idx0 = j*N + res
            idx1= idx0+steps+1-j
            idx = np.arange(idx0, idx1)%N + j*N
            mask[idx] = 1             
    return mask 
    
def action_mask(state, N):
    "Mask of the actions that can be performed"
    return np.append(contained_constraints(state, N), False) == False

## Useful mathematical operations

In [ ]:
#export    
def dist_exp(xmin, x):
    """This function computes an eponential distance given a minimum value (in terms of absolute value)."""
    return np.exp(1-x/xmin)

def dist_poly(x, xmin, xmax, d):
    """This function computes a linear decay bound from 1 to 0 given a minimum and maximum value (in terms of 
    absolute value). Exponentiate the line to get better shapes"""
    if np.allclose(xmin, xmax): x[:] = 1.; return x 
    else:                       return ((x-xmax)/(xmin-xmax))**d 
    
def binomial(n, k):
    return np.math.factorial(n)/(np.math.factorial(k)*np.math.factorial(n-k))

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_environment.ipynb.
Converted 01_agents.ipynb.
Converted 02_budget_profiles.ipynb.
Converted 03_hamiltonian.ipynb.
Converted 04_training.ipynb.
Converted 05_utils.ipynb.
Converted 06_sdp.ipynb.
Converted index.ipynb.
